In [1]:
! sudo cp /home/jovyan/work/jars/neo4j-connector-apache-spark_2.12-4.1.0_for_spark_3.jar /usr/local/spark/jars/neo4j-connector-apache-spark_2.12-4.1.0_for_spark_3.jar

In [3]:
import pyspark
from pyspark.sql import SparkSession
# NEO4J  CONFIGURATION
bolt_url = "bolt://neo4j:7687"
# Spark init
spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-pyspark') \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

5.	Load a Spark dataframe of USA country suppliers and their products for products that are not discontinued. Include supplier company and contact names, country and phone. From products include product name, discontinued, unit price and units in stock.

In [4]:
cq = '''
match (s:Supplier)-[:SUPPLIES]->(p:Product)
where s.country = 'USA' and p.discontinued=false
return s.companyName, s.contactName, s.country, p.productName, p.discontinued, p.unitPrice, p.unitsInStock
'''
df = spark.read.format("org.neo4j.spark.DataSource").option("url",bolt_url).option("query",cq).load()

In [6]:
df.toPandas()

,s.companyName,s.contactName,s.country,p.productName,p.discontinued,p.unitPrice,p.unitsInStock
0,New Orleans Cajun Delights,Shelley Burke,USA,Chef Anton's Cajun Seasoning,False,22.00,53
1,New Orleans Cajun Delights,Shelley Burke,USA,Louisiana Fiery Hot Pepper Sauce,False,21.05,76
2,New Orleans Cajun Delights,Shelley Burke,USA,Louisiana Hot Spiced Okra,False,17.00,4
3,Grandma Kelly's Homestead,Regina Murphy,USA,Uncle Bob's Organic Dried Pears,False,30.00,15
4,Grandma Kelly's Homestead,Regina Murphy,USA,Northwoods Cranberry Sauce,False,40.00,6
5,Grandma Kelly's Homestead,Regina Murphy,USA,Grandma's Boysenberry Spread,False,25.00,120
6,Bigfoot Breweries,Cheryl Saylor,USA,Sasquatch Ale,False,14.00,111
7,Bigfoot Breweries,Cheryl Saylor,USA,Steeleye Stout,False,18.00,20
8,Bigfoot Breweries,Cheryl Saylor,USA,Laughing Lumberjack Lager,False,14.00,22
9,New England Seafood Cannery,Robb Merchant,USA,Jack's New England Clam Chowder,False,9.65,85


In [21]:
from pyspark.sql.functions import col, concat, lit
emps = spark.read.option("multiline",True).json("file:///home/jovyan/datasets/fudgemart/fudgemart-employees.json") \
.withColumn("employee_name", concat(col("employee_firstname"), lit(" "), col("employee_lastname")) )

In [22]:
emps.toPandas()

,employee_birthdate,employee_department,employee_firstname,employee_fulltime,employee_hiredate,employee_hourlywage,employee_id,employee_jobtitle,employee_lastname,employee_ssn,employee_supervisor_id,employee_termdate,employee_name
0,1982-01-12T00:00:00,Electronics,Arial,0,2011-07-05T00:00:00,15.2830,1,Sales Associate,Photo,111220001,5.0,None,Arial Photo
1,1982-11-30T00:00:00,Electronics,Sal,1,2005-07-26T00:00:00,15.0699,2,Sales Associate,Ladd,111220002,5.0,None,Sal Ladd
2,1972-09-03T00:00:00,Hardware,Dustin,0,2004-07-02T00:00:00,12.4500,3,Sales Associate,Dawind,111220003,6.0,2010-11-06T00:00:00,Dustin Dawind
3,1990-05-13T00:00:00,Hardware,Sandi,1,2011-01-02T00:00:00,13.0997,4,Sales Associate,Shores,111220004,6.0,None,Sandi Shores
4,1974-02-22T00:00:00,Electronics,Isabelle,1,2005-08-16T00:00:00,20.1287,5,Department Manager,Gunnering,111220005,32.0,None,Isabelle Gunnering
5,1973-07-29T00:00:00,Hardware,Lee,1,2004-01-26T00:00:00,18.5845,6,Department Manager,Hvmeehom,111220006,32.0,None,Lee Hvmeehom
6,1988-04-03T00:00:00,Housewares,Allan,1,2005-02-22T00:00:00,12.0878,7,Sales Associate,Wrench,111220007,24.0,None,Allan Wrench
7,1973-12-13T00:00:00,Sporting Goods,Ally,1,2005-08-14T00:00:00,12.3541,8,Sales Associate,Gator,111220008,27.0,None,Ally Gator
8,1980-11-30T00:00:00,Housewares,Alma,1,2004-11-12T00:00:00,11.7152,9,Sales Associate,Frienzergon,111220009,24.0,None,Alma Frienzergon
9,1976-10-02T00:00:00,Hardware,Artie,1,2005-04-04T00:00:00,12.7269,10,Sales Associate,Choke,111220010,6.0,None,Artie Choke


In [26]:
cql = '''
MERGE (e:Employee { name: event.employee_name, dept: event.employee_department, 
                    jobtitle: event.employee_jobtitle, id: event.employee_id })
'''
tmp = emps.select("employee_id", "employee_name", "employee_jobtitle", "employee_department")
tmp.write.format("org.neo4j.spark.DataSource").mode("Overwrite") \
  .option("url", bolt_url) \
  .option("query",cql) \
  .save()

In [32]:
cql = '''
MATCH (e:Employee)
MATCH (s:Employee)
where e.id = event.employee_id and s.id = event.employee_supervisor_id
MERGE (s)-[:SUPERVISES]->(e)
'''

tmp = emps.select("employee_supervisor_id", "employee_id").orderBy("employee_supervisor_id")
tmp.write.format("org.neo4j.spark.DataSource").mode("Overwrite") \
  .option("url", bolt_url) \
  .option("query",cql) \
  .save()